<a href="https://colab.research.google.com/github/Stonky-Boi/GDSC_Hackathon_410CODERS/blob/main/410_Coders_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install cmake (needed for building dlib) and other dependencies
!apt-get install -y cmake

# Install dlib and opencv-python
!pip install dlib opencv-python
!pip install gradio
!pip uninstall -y numpy
!pip install numpy==1.23.5

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.0 requires numpy>=1

In [1]:
import cv2
import dlib
import numpy as np
import gradio as gr

# Initialize dlib's face detector and shape predictor (ensure the .dat file is available)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def get_landmarks(img, detector, predictor):
    """Detect face and extract 68 landmarks from a given image."""
    # Ensure image is in uint8
    if img.dtype != np.uint8:
        img = np.uint8(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        return None
    # Process first detected face
    face = faces[0]
    landmarks = predictor(gray, face)
    points = []
    for i in range(68):
        points.append((landmarks.part(i).x, landmarks.part(i).y))
    return points

def apply_affine_transform(src, src_tri, dst_tri, size):
    """Apply affine transform calculated using src_tri and dst_tri to src image."""
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    dst = cv2.warpAffine(src, warp_mat, (size[0], size[1]), flags=cv2.INTER_LINEAR,
                         borderMode=cv2.BORDER_REFLECT_101)
    return dst

def warp_triangle(img1, img2, t1, t2):
    """Warp and blend triangular regions from img1 to img2."""
    # Find bounding rectangles for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))

    # Offset points by top left corner of the respective rectangles
    t1_rect = []
    t2_rect = []
    t2_rect_int = []
    for i in range(3):
        t1_rect.append(((t1[i][0] - r1[0]), (t1[i][1] - r1[1])))
        t2_rect.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
        t2_rect_int.append((int(t2[i][0] - r2[0]), int(t2[i][1] - r2[1])))

    # Create a mask for the triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.array(t2_rect_int), (1.0, 1.0, 1.0), 16, 0)

    # Apply affine transform to the small rectangular patch
    img1_rect = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
    size_rect = (r2[2], r2[3])
    img2_rect = apply_affine_transform(img1_rect, t1_rect, t2_rect, size_rect)

    # Blend the warped triangle into the destination image
    img2_rect = img2_rect * mask
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = \
        img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ((1.0, 1.0, 1.0) - mask)
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] += img2_rect

def face_swap(source, target):
    """
    Perform face swapping using a source face and a target image.
    The inputs (source, target) are expected as numpy arrays in RGB format.
    """
    # Convert RGB (Gradio default) to BGR (OpenCV default)
    source_bgr = cv2.cvtColor(source, cv2.COLOR_RGB2BGR)
    target_bgr = cv2.cvtColor(target, cv2.COLOR_RGB2BGR)

    # Get landmarks from both images
    points1 = get_landmarks(source_bgr, detector, predictor)
    points2 = get_landmarks(target_bgr, detector, predictor)
    if points1 is None or points2 is None:
        return "Face not detected in one of the images!"

    # Convert images to float32 for processing
    img1 = np.float32(source_bgr)
    img2 = np.float32(target_bgr)

    # Get convex hull for target face landmarks
    hullIndex = cv2.convexHull(np.array(points2), returnPoints=False)
    hull1 = []
    hull2 = []
    for i in range(len(hullIndex)):
        idx = hullIndex[i][0]
        hull1.append(points1[idx])
        hull2.append(points2[idx])

    # Delaunay triangulation on the target face
    sizeImg2 = img2.shape
    rect = (0, 0, sizeImg2[1], sizeImg2[0])
    subdiv = cv2.Subdiv2D(rect)
    for p in hull2:
        subdiv.insert(p)
    triangleList = subdiv.getTriangleList()
    triangles = []
    for t in triangleList:
        pts = [(t[0], t[1]), (t[2], t[3]), (t[4], t[5])]
        indices = []
        for pt in pts:
            for i in range(len(hull2)):
                if abs(pt[0] - hull2[i][0]) < 1.0 and abs(pt[1] - hull2[i][1]) < 1.0:
                    indices.append(i)
        if len(indices) == 3:
            triangles.append(indices)

    # Warp triangles from source face to target image
    img2_warped = np.copy(img2)
    for tri in triangles:
        t1 = [hull1[tri[0]], hull1[tri[1]], hull1[tri[2]]]
        t2 = [hull2[tri[0]], hull2[tri[1]], hull2[tri[2]]]
        warp_triangle(img1, img2_warped, t1, t2)

    # Create mask from convex hull on target face and perform seamless cloning
    hull8U = [(p[0], p[1]) for p in hull2]
    mask = np.zeros(img2.shape, dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(hull8U), (255, 255, 255))

    r = cv2.boundingRect(np.float32([hull2]))
    center = (int(r[0] + r[2] / 2), int(r[1] + r[3] / 2))

    output = cv2.seamlessClone(np.uint8(img2_warped), np.uint8(img2), mask, center, cv2.NORMAL_CLONE)

    # Convert output back to RGB for Gradio display
    output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
    return output_rgb

# Create a Gradio interface.
iface = gr.Interface(
    fn=face_swap,
    inputs=[
        gr.Image(type="numpy", label="Source Face"),
        gr.Image(type="numpy", label="Target Image")
    ],
    outputs=gr.Image(type="numpy", label="Face Swapped Output"),
    title="Face Swapping App",
    description="Upload a source face and a target image. The app will swap the face from the source into the target."
)

# Launch the Gradio app
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d4bcc4aadcb4ae815d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
